# Simple NN


In [2]:
import sys
# sys.path.append("/home/arseni1919/PycharmProjects/NT_sandbox")
sys.path.append("/Users/arseniperchik/PycharmProjects/NT_sandbox")

print(sys.path)
from IMPORTS import *

%matplotlib inline
%config Completer.use_jedi = False
# print(sys.path)

['/Users/arseniperchik/PycharmProjects/NT_sandbox/NN', '/Users/arseniperchik/opt/anaconda3/envs/DRL4/lib/python37.zip', '/Users/arseniperchik/opt/anaconda3/envs/DRL4/lib/python3.7', '/Users/arseniperchik/opt/anaconda3/envs/DRL4/lib/python3.7/lib-dynload', '', '/Users/arseniperchik/.local/lib/python3.7/site-packages', '/Users/arseniperchik/opt/anaconda3/envs/DRL4/lib/python3.7/site-packages', '/Users/arseniperchik/PycharmProjects/gym-stocktrading', '/Users/arseniperchik/opt/anaconda3/envs/DRL4/lib/python3.7/site-packages/IPython/extensions', '/Users/arseniperchik/.ipython', '/Users/arseniperchik/PycharmProjects/NT_sandbox']


# Some Function - *y_func*

In [72]:
SCALE = 50.0
x = np.linspace(0.0, SCALE, num=100)
y = np.linspace(0.0, SCALE, num=100)
z = np.zeros((len(x),len(y)))

def y_func(a, b):
    return np.sin( a / 5) * np.cos( b / 5)/2 + 0.5# - np.log(b)
    
for i_indx, i in enumerate(x):
    for j_indx, j in enumerate(y):
#         z[i,j] = np.log(i) + np.log(j)
        z[i_indx,j_indx] = y_func(i, j)
#         z[i_indx,j_indx] = np.sin(10*(i**2+j**2))/10.0

fig = go.Figure(
    data=go.Surface(z=z)
)
fig

## I - Dataset 

In [104]:
import random
REPLAY_SIZE = 10000
class ALGDataset(Dataset):
    def __init__(self, func):
        self.func = func
        self.buffer = deque(maxlen=REPLAY_SIZE)
        for i in range(REPLAY_SIZE):
            x_num = random.uniform(0, SCALE)
            y_num = random.uniform(0, SCALE)
            z_num = self.func(x_num, y_num)
            obs_x = torch.Tensor([x_num/100.0, y_num/100.0])
            obs = (obs_x, z_num)
            self.buffer.append(obs)

    def __len__(self):
        return len(self.buffer)

    def __getitem__(self, indx):
        z_num = self.buffer[indx]
        return z_num
    
    def append(self, new_z):
        self.buffer.append(new_z)

alg_dataset = ALGDataset(y_func)


## II - LightningDataModule

In [105]:
class ALGDataModule(pl.LightningDataModule):

    def __init__(self, alg_dataset):
        super().__init__()
        self.alg_dataset=alg_dataset

    def setup(self, stage=None):
        pass

    def train_dataloader(self):
        return DataLoader(self.alg_dataset, batch_size=64)

    def val_dataloader(self):
        pass

    def test_dataloader(self):
        pass

data_module = ALGDataModule(alg_dataset)

In [106]:
# for batch in data_module.train_dataloader():
#     print(batch)

## III - ALGLightningModule

In [116]:
class ALGLightningModule(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 256),
            nn.ReLU(),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Linear(256, 256),
#             nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            # nn.Linear(128, 128),
            # nn.ReLU(),
            nn.Linear(128, 1),
#             nn.ReLU(),
        )
#         self.l1 = torch.nn.Linear(2, 100)
#         self.l2 = torch.nn.Linear(100, 1)
#         self.l3 = torch.nn.Linear(100, 1)
        self.counter = 0

    def forward(self, x):
#         return self.l2(torch.relu(self.l1(x)))
#         x = F.log_softmax(self.net(x), dim=1)
        x = self.net(x)
        return x

    def training_step(self, batch, batch_idx):
        print(f'\r{self.counter}', end='')
        self.counter += 1
        x, y = batch
#         print(f'x size: {x.size()}')
#         print(f'y reshape: {y.reshape(-1).size()}')
#         print(f'X: {x.float().dtype}')
#         print(f'Y: {y.float().dtype}')
        
        y_hat = self(x.float())
        loss = F.mse_loss(y_hat, y.float())
#         loss = F.cross_entropy(y_hat, y)

        self.log('train loss', loss)
        # if batch_idx % 1000 == 0:
        #     print(epsilon)
        
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=LR)
    
    def train_dataloader(self):
        return DataLoader(alg_dataset, batch_size=64)

alg_module = ALGLightningModule()

In [117]:

# PARAMS = {'LR': LR}
# neptune_logger = NeptuneLogger(
#     api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOTBjYmViMGYtN2Q0ZC00NWEyLWE3OTItM2NlODg0Yjc4M2M0In0=",
#     project_name="1919ars/NA-sandbox",
#     params=PARAMS)

In [118]:
# from torchvision.datasets import MNIST
# train_loader = DataLoader(MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()))
# model = ALGLightningModule(input_size=28 * 28, output_size=10)
# trainer = pl.Trainer(logger=neptune_logger)

trainer = pl.Trainer(max_epochs=20)
# trainer.fit(model=alg_module, datamodule=data_module)
trainer.fit(model=alg_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 33 K  


/Users/arseniperchik/opt/anaconda3/envs/DRL4/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning:

Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



179

/Users/arseniperchik/opt/anaconda3/envs/DRL4/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning:

Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



3139


1

In [120]:
SCALE = 50.0
x = np.linspace(0.0, SCALE, num=100)
y = np.linspace(0.0, SCALE, num=100)
z_hat = np.zeros((len(x),len(y)))
z_real = np.zeros((len(x),len(y)))
    
for i_indx, i in enumerate(x):
    for j_indx, j in enumerate(y):
        z_hat[i_indx,j_indx] = alg_module(torch.Tensor([[i/100.0, j/100.0]])).item()
        z_real[i_indx,j_indx] = y_func(i, j)
         

fig = go.Figure(
    data=[
        go.Surface(z=z_hat),
        go.Surface(z=z_real, opacity=0.5)
    ]
)
# showscale=False, colorbar_x=-0.07,
fig

In [115]:
alg_module(torch.Tensor([[1000, 0.2]])).item()

0.0